In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [3]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(file_path='results.csv')

docs = loader.load()

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)

vector_store = Chroma.from_documents(documents = splits, embedding = OpenAIEmbeddings())

In [6]:
retriever = vector_store.as_retriever()




In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
                                          Answer the following questions based only on the provided context.
                                          think step by step before before providing a detailed answer.
                                          <context>
                                          {context}
                                          </context>
                                          Question: {input}
                                          """)

In [8]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [17]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)


In [19]:
from langchain.chains import create_retrieval_chain

retrieveal_chain = create_retrieval_chain(retriever, document_chain)

In [20]:
response = retrieveal_chain.invoke({"input": "Who won the Copa America 2024?"})
print("Generated Text:", response)


Generated Text: {'input': 'Who won the Copa America 2024?', 'context': [Document(page_content='date: 2024-06-29\nhome_team: Canada\naway_team: Chile\nhome_score: 0\naway_score: 0\ntournament: Copa América\ncity: Orlando\ncountry: United States\nneutral: TRUE', metadata={'row': 23340, 'source': 'results.csv'}), Document(page_content='date: 2024-07-02\nhome_team: Costa Rica\naway_team: Paraguay\nhome_score: 2\naway_score: 1\ntournament: Copa América\ncity: Austin\ncountry: United States\nneutral: TRUE', metadata={'row': 23354, 'source': 'results.csv'}), Document(page_content='date: 2024-06-21\nhome_team: Peru\naway_team: Chile\nhome_score: 0\naway_score: 0\ntournament: Copa América\ncity: Arlington\ncountry: United States\nneutral: TRUE', metadata={'row': 23303, 'source': 'results.csv'}), Document(page_content='date: 2024-06-24\nhome_team: Brazil\naway_team: Costa Rica\nhome_score: 0\naway_score: 0\ntournament: Copa América\ncity: Inglewood\ncountry: United States\nneutral: TRUE', metada